In [1]:
import warnings
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Suppress annoying harmless error.
warnings.filterwarnings(action="ignore")


In [2]:
y2015 = pd.read_csv(
    'LoanStats3d.csv',
    skipinitialspace=True,
    header=1
)

In [ ]:
y2015.head()

I believe the original data I saw (for a brief second until the kernal crashed and I was unable to reload the data) had 111 columns and actually had id information

In [ ]:
categorical = y2015.select_dtypes(include=['object'])

for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

In [3]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [4]:
y2015 = y2015[:-2]

In [5]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)


In [6]:
rfc.fit(X, Y)

# get feature importances
frank=rfc.feature_importances_
frankdf = pd.DataFrame(frank, index=X.columns, columns=['Importance']).sort_values('Importance', ascending=False)

print(frankdf.head(10))

                         Importance
out_prncp_inv              0.196462
recoveries                 0.122315
out_prncp                  0.120141
collection_recovery_fee    0.061705
total_rec_prncp            0.060442
last_pymnt_d_Sep-2018      0.052286
next_pymnt_d_Oct-2018      0.048439
last_pymnt_amnt            0.043433
total_pymnt                0.032642
term_ 60 months            0.027453


In [7]:
from sklearn.decomposition import PCA
sklearnPCA = PCA(n_components = 4)
transformed = sklearnPCA.fit_transform(X)

features = sklearnPCA.explained_variance_ratio_ 
print(features)

#print(sklearnPCA.components_)

[0.83722452 0.06901664 0.04370485 0.02703175]


In [8]:
cross_val_score(rfc, transformed, Y, cv=10)

array([0.54381174, 0.53953742, 0.54108093, 0.5376505 , 0.53873189,
       0.53889812, 0.54206939, 0.54131142, 0.542868  , 0.54517171])

In [9]:
pca = PCA(n_components = 15)
x1 = pca.fit_transform(X)
cross_val_score(rfc, x1, Y, cv=10)

array([0.95383739, 0.94913564, 0.93619396, 0.92429531, 0.88793636,
       0.90102114, 0.96019853, 0.97112119, 0.97033677, 0.97040802])

As seen in the dataframe above of the feature ranks we can select the most important features to build the model on.  Using that list would (hopefully) allow us to drop the payment amount and outstanding principal and still maintain a cross validation of larger than .90.  

This data set is too large to continue to play around with.